Перед началом прохождения убедитесь, что все настроено как надо. Во-первых, **перезапустите ядро** (**restart the kernel**) (в панели меню выберите Kernel $\rightarrow$ Restart) и затем **запуск всех ячеек** (**run all cells**) (в панели меню выберите Cell $\rightarrow$ Run All).

Убедитесь, что вы заменили всем такие места 
```python
# YOUR CODE HERE
raise NotImplementedError()
```
и
```
"YOUR ANSWER HERE"
```
на свои решения.

> Не переименовывайте ноутбук!

Также, заполните свои данные (имя и группу):

In [1]:
NAME = "Вячеслав"
GROUP = "6492"

In [2]:
import uuid

STUDENT_UUID = uuid.uuid4()
print(f"Your uuid is {STUDENT_UUID}")

Your uuid is 9e57c57e-587b-41d5-ba78-a17639af5410


---

# И снова здравствуйте!

Это третья контрольная работа с некоторыми вопросами по теме классификации в машинном обучении!

Если у вас не установлены какие-то пакеты, то посмотрите нужные версии в ноутбуках по классификации (20-23).

Не забывайте о следующих особенностях:
- Помимо указанных тестов после задания существуют и скрытые тесты, поэтому внимательнее читайте задание
- Обязательно проверяйте выполнимость на тестах, для проверки перезапустите ядро и запустите все ячейки заново
- После скачивания ноутбуков достаточно заполнить места для ответов и отправить ноутбук обратно

In [3]:
import numpy as np
print(f"numpy version: {np.__version__}")
import pandas as pd
print(f"pandas version: {pd.__version__}")
# Just to check that it exists
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

from sklearn.datasets import load_iris, load_linnerud

from typing import Tuple, List, Union

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

numpy version: 1.21.2
pandas version: 1.3.3
scikit-learn version: 0.24.2


### Задание 1

У вас имеется следующий код и разработчик дает следующее описание:

"Я сделал код для датасета linnerud, который выбирает примеры со значением прыжков меньше 80. Написал как смог =)"

```python
valid_data = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if j == 2:
            row = data.iloc[i] 
            value = row[2]
            if value < 80:
                valid_data.append(row.name)
```

Ваша задача - написать код, который выбирает примеры с количеством приседаний от 120 до 250 (не включительно). Для реализации необходимо использовать pandas.

In [4]:
def extract_average_situps_indexes(data: pd.DataFrame) -> list:
    mask = (data['Situps']>120) & (data['Situps']<250)
    return data[mask].index

In [5]:
features, target = load_linnerud(return_X_y=True, as_frame=True)

indexes = extract_average_situps_indexes(features)
np.testing.assert_array_equal(indexes, [0, 4, 7, 8, 11, 12, 15, 17, 18])


### Задание 2

Напишите класс классификатора, который проверяет значение задаваемой колонки (индекса по порядку) (anomaly_col_idx) и для примеров со значением больше заданного порога (threshold) **по абсолютному значению** записывает в предсказание "Anomaly!". Для предсказания используйте LogisticRegression и не забудьте зафиксировать random_state (для этого есть RANDOM_SEED переменная)!


In [6]:
from sklearn.linear_model import LogisticRegression
class ClassifierWithAnomalyDetection:
    def __init__(self, anomaly_col_idx: int, threshold: float):
        self.idx = anomaly_col_idx
        self.threshold = threshold

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        self.model = LogisticRegression(random_state=RANDOM_SEED)
        self.model.fit(X_train, y_train)

    def predict(self, X_test: pd.DataFrame):
        mask = abs(X_test.iloc[:,self.idx])>self.threshold
        y_pred = self.model.predict(X_test)
        for i in range(len(mask)):
            if mask[mask.index[i]] == True:
                y_pred[i] = "Anomaly!"
        return y_pred
        

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Prepare data for testing
X_data, y_data = make_classification(
    n_samples=100, 
    n_features=2, 
    n_redundant=0,
    scale=[1, 10],
    random_state=RANDOM_SEED)

train_columns = ["Feature_1", "Feature_2"]
target_column = "target"

df = pd.DataFrame(X_data, columns=train_columns)
df[target_column] = pd.Series(y_data).map({0: "bad", 1: "good"})

train_df, test_df = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)

classifier = ClassifierWithAnomalyDetection(anomaly_col_idx=0, threshold=18)
    
classifier.fit(train_df[train_columns], train_df[target_column])
y_pred = classifier.predict(test_df[train_columns])
y_true = test_df[target_column]

assert "Anomaly!" in y_pred
assert "bad" in y_pred
assert "good" in y_pred

np.testing.assert_equal(len(y_pred), len(test_df))


### Задание 3

Перепешите реализацию кода объединения колонок с использованием pandas:

```python
df_1_data = df_main.values
df_2_data = df_noise.values

new_column = [] 
for i in range(df_1_data.shape[0]):
    for j in range(df_1_data.shape[1]):
        if j == 0:
            for k in range(df_2_data.shape[0]):
                for m in range(df_2_data.shape[1]):
                    if m == 0:
                        if df_1_data[i][j] == df_2_data[k][m]:
                            for n in range(df_2_data.shape[1]):
                                if n == 1:
                                    new_column.append(df_2_data[k][n])
df_main = df_main.copy()
df_main['has_noise'] = new_column

return df_main
```

In [ ]:
def append_noise_column(df_main: pd.DataFrame, df_noise: pd.DataFrame) -> pd.DataFrame:
    return df_main.merge(df_noise, on = 'id', how = 'left')

In [ ]:
df_main = pd.DataFrame(
    [
        [1, 1.5], [1, 2.3], 
        [2, 0.7], [2, 1.6], [2, 3.4], 
        [3, 0.5], [3, 0.5], [3, 0.5]
    ], 
    columns=["id", "value"]
)
df_noise = pd.DataFrame(
    [
        [1, True], [2, False], [3, False]
    ], 
    columns=["id", "has_noise"]
)

df_with_noise = append_noise_column(df_main, df_noise)

np.testing.assert_array_equal(df_with_noise.columns, ["id", "value", "has_noise"])

np.testing.assert_equal(df_with_noise.loc[0, "has_noise"], True)
np.testing.assert_equal(df_with_noise.loc[4, "has_noise"], False)
np.testing.assert_equal(df_with_noise.loc[7, "has_noise"], False)


### Задание 4

Найдите индексы строк, где колонка делится на n_div без остатка. Например, значения wine_serving на 3.

In [ ]:
def get_n_divisible_indexes(df: pd.DataFrame, column_name: str, n_div: int) -> list:
    mask = (df[column_name] % n_div) == 0
    df = df[mask]
    return df.index

In [ ]:
features, target = load_linnerud(return_X_y=True, as_frame=True)

indexes = get_n_divisible_indexes(features, "Jumps", 5)
np.testing.assert_array_equal(indexes, [0,  1,  7,  8,  9, 11, 12, 13, 15, 16, 17])


### Задание 5

Напишите функцию, которая строит confusion matrix (true ~ rows, pred ~ cols) по заданным классам по осям. Остальные классы скидываются в объединенный класс "Другое".

Например, имеем:

```python
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]
```

Мы задаем целевые классы

```python
true_classes = [0, 1]
pred_classes = [1, 2]
```

Должна получиться матрица (DataFrame) с соответсвующими именами строк и колонок.

|    |  1 | 2  | Другое |
|---:|:--:|:--:|:--:  |
| 0  | 1  | 0  |    0    |
| 1  | 3  | 0  |   1     |
|Другое| 0 | 1   |   2     |


In [ ]:
def custom_confusion_matrix(y_true: list, y_pred: list, true_classes: list, pred_classes:list) -> pd.DataFrame:
    true_classes.append("Другое")
    pred_classes.append("Другое")
    data = np.zeros(shape = (len(true_classes), len(pred_classes)))
    df = pd.DataFrame(data, index= true_classes ,columns= pred_classes)

    def classes (i, classesValue):
        if i in classesValue:
            return i
        else:
            return "Другое"

    for i in range(len(y_true)):
        y_true[i] = classes(y_true[i], true_classes)
        y_pred[i] = classes(y_pred[i], pred_classes)
        df.loc[y_true[i], y_pred[i]] +=1

    return df

In [ ]:
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]

true_classes = [0, 1]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
assert isinstance(conf, pd.DataFrame)
np.testing.assert_array_equal(conf.columns, np.array([1, 2, "Другое"], dtype=object))
np.testing.assert_array_equal(conf.index, np.array([0, 1, "Другое"], dtype=object))
np.testing.assert_equal(conf.loc[1, 1], 3)
np.testing.assert_equal(conf.loc["Другое", 2], 1)

In [ ]:
# Another test
y_true = [0, 1, 1, 2, 2, 3, 1, 1, 2, 1, 0]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0, 4, 1, 2]

true_classes = [0, 3]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
np.testing.assert_array_equal(conf, [[1, 1, 0], [0, 0, 1], [4, 1, 3]]) 



### Задание 6

Напишите функцию, которая принимает вектор предсказаний и возвращает индексы примеров, в которых модель совершила ошибки. Обратите внимание, что входной вектор может быть как обычным list или tuple, так и pd.Series. Вернуть необходимо индексы (index) из исходного y_true.

In [ ]:
def get_error_indexes(y_true: pd.Series, y_pred: Union[list, tuple, pd.Series]) -> pd.DataFrame:
    arr=[]
    for i in range(len(y_true)):
        if y_true[y_true.index[i]]!=y_pred[i]:
            arr.append(y_true.index[i])
    return arr

In [ ]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[50:60]
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 1, 1]

err_idxs = get_error_indexes(y_test, y_pred)
np.testing.assert_array_equal(err_idxs, [50, 51, 55, 56, 57])


In [ ]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[70:80]
y_pred = pd.Series([0, 1, 0, 1, 0, 1, 1, 0, 0, 1])

err_idxs = get_error_indexes(y_test, y_pred)

np.testing.assert_array_equal(err_idxs, [70, 72, 74, 77, 78])
